In [1]:
#Imports and TF version check
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
import numpy as np


TEST = False
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


In [2]:
#Loading the vgg16 model
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    if(sess is None):
        sess = tf.Session()
        print('Created new session')
    graph = tf.get_default_graph()
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    model_exists = tf.saved_model.loader.maybe_saved_model_directory(vgg_path)
    model = []
    if(model_exists):
        print("Model exists")
        tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)
    else:
        print("No model in directory")
        return
   
    with tf.name_scope("Vgg16_encoder"):
        input_image = graph.get_tensor_by_name(vgg_input_tensor_name)
        keep_prob =  graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
        layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
        layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
        layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)

    
    return input_image, keep_prob, layer3_out, layer4_out, layer7_out

helper.maybe_download_pretrained_vgg("./data")
vgg_path = './data/vgg/'
sess = None
if(TEST):
    tests.test_load_vgg(load_vgg, vgg_path, tf)


In [3]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer7_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer3_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    
    std = 1e-2
    
    with tf.name_scope('1x1_convolution'):
        conv7_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, kernel_size = 1, strides = 1, padding = 'same',
                                    kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3), name = 'conv7_1x1',
                                    kernel_initializer=helper.truncated_norm(std))
    
    pool4_1x1 = tf.layers.conv2d(vgg_layer4_out, num_classes, kernel_size = 1, strides = 1, padding = 'same',
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3), name = 'pool4_1x1',
                                kernel_initializer=helper.truncated_norm(std))
    
    pool3_1x1 = tf.layers.conv2d(vgg_layer3_out, num_classes, kernel_size = 1, strides = 1, padding = 'same',
                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3), name = 'pool3_1x1',
                                kernel_initializer=helper.truncated_norm(std))
    
    with tf.name_scope('Upsampleby2_1'):
        upsample_2_conv7 = tf.layers.conv2d_transpose(conv7_1x1, num_classes , kernel_size = 4, strides = 2 ,padding='same',
                                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3), name = 'upsampleX2_conv7')

    skip_pool4_conv7 = tf.add(upsample_2_conv7, pool4_1x1,name = 'skip_pool4_conv7')
    
    with tf.name_scope('Upsampleby2_2'):
        upsample_2_skip1 = tf.layers.conv2d_transpose(skip_pool4_conv7, num_classes, kernel_size = 4, strides = 2, padding='same',
                                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),name = 'upsampleX2_skip_pool4_conv7',
                                                     kernel_initializer=helper.truncated_norm(std))

    skip_pool3 = tf.add(upsample_2_skip1,pool3_1x1,name = 'skip_pool3' )
    
    with tf.name_scope('Upsampleby8'):
        out = tf.layers.conv2d_transpose(skip_pool3, num_classes, kernel_size = 16, strides = 8, padding='same',
                                                kernel_regularizer = tf.contrib.layers.l2_regularizer(1e-3),name = 'upsampleX8_skip_pool3',
                                        kernel_initializer=helper.truncated_norm(std))

    
    return out
if(TEST):
    tests.test_layers(layers)

In [4]:

def optimize(nn_last_layer, correct_labels, learn_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer,(-1,num_classes), name = 'Logits')
    correct_labels = tf.reshape(correct_labels,(-1,num_classes), name = 'Labels')
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels = correct_labels, name = 'LossFn'), name = 'Loss_mean')
    optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate, name = 'Optimizer').minimize(loss, name = 'train_op')
    
    return logits, optimizer, loss
if(TEST):
    tests.test_optimize(optimize)

In [5]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_labels, keep_prob, learning_rate,lr,kprob):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    tf.logging.set_verbosity(tf.logging.INFO)
    sess.run(tf.global_variables_initializer())
    tf.summary.scalar("Training_Loss", cross_entropy_loss)
    #tf.summary.scalar('Learning_rate',learning_rate)
    tf.summary.image('Images', input_image)
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    merged_summ = tf.summary.merge_all()
    summary_dir = './summaries'
    train_writer = tf.summary.FileWriter(summary_dir + '/train',sess.graph)
    # TODO: Implement function
    step = 0
    for epoch in range(epochs):
        for i, (batch, batch_labels) in enumerate(get_batches_fn(batch_size)):
            step+=1
            feed = {keep_prob : kprob, learning_rate : lr, input_image : batch, correct_labels : batch_labels}
            if(step %55 != 0):
                _,loss_hist,summary = sess.run([train_op,cross_entropy_loss,merged_summ], feed_dict = feed)
                train_writer.add_summary(summary, step)
            else:
                _,loss_hist,summary = sess.run([train_op,cross_entropy_loss,merged_summ], feed_dict = feed, options=run_options,  run_metadata=run_metadata)
                train_writer.add_summary(summary, step)
                train_writer.add_run_metadata(run_metadata, 'Step%d' % step)
        print('Epoch' + str(epoch+1) + '/' + str(epochs) + " Loss: " + str(loss_hist))
    
    pass
if(TEST):
    tests.test_train_nn(train_nn)

In [6]:

def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    if(TEST):
        tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/
    
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)
        
        
        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network
        
        # TODO: Build NN using load_vgg, layers, and optimize function
        learning_rate = tf.placeholder(tf.float32)
        lr = 1e-4
        kprob = 0.5
        epochs = 50
        batch_size = 5
        out_shape = (None,None,None,num_classes)
        
        stub_img = np.zeros((1,160,576,3))
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        sh= tf.shape(layer7_out)
        correct_labels = tf.placeholder(tf.float32,out_shape)
        output = layers(layer3_out, layer4_out, layer7_out, num_classes)
        #print(sess.run([conv1],feed_dict = {keep_prob:1,input_image:stub_img}))
        logits, optimizer, loss = optimize(output, correct_labels, learning_rate, num_classes)
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, optimizer, loss, input_image,
             correct_labels, keep_prob = keep_prob, learning_rate = learning_rate,lr= lr,kprob = kprob)
        print('finished training')
        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        # OPTIONAL: Apply the trained model to a video


if __name__ == '__main__':
    run()

Model exists
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Epoch1/50 Loss: 0.219209
Epoch2/50 Loss: 0.194928
Epoch3/50 Loss: 0.157736
Epoch4/50 Loss: 0.0930718
Epoch5/50 Loss: 0.101493
Epoch6/50 Loss: 0.0542723
Epoch7/50 Loss: 0.0681034
Epoch8/50 Loss: 0.101767
Epoch9/50 Loss: 0.0499339
Epoch10/50 Loss: 0.0643152
Epoch11/50 Loss: 0.0452011
Epoch12/50 Loss: 0.0688554
Epoch13/50 Loss: 0.0437873
Epoch14/50 Loss: 0.0414476
Epoch15/50 Loss: 0.0382417
Epoch16/50 Loss: 0.0467688
Epoch17/50 Loss: 0.0258814
Epoch18/50 Loss: 0.0303439
Epoch19/50 Loss: 0.0401812
Epoch20/50 Loss: 0.0341045
Epoch21/50 Loss: 0.0327011
Epoch22/50 Loss: 0.0373522
Epoch23/50 Loss: 0.0332897
Epoch24/50 Loss: 0.0227356
Epoch25/50 Loss: 0.0191436
Epoch26/50 Loss: 0.0276415
Epoch27/50 Loss: 0.0233207
Epoch28/50 Loss: 0.0166718
Epoch29/50 Loss: 0.0218343
Epoch30/50 Loss: 0.0264237
Epoch31/50 Loss: 0.0264885
Epoch32/50 Loss: 0.0140731
Epoch33/50 Loss: 0.0228715
Epoch34/50 Loss: 0.0237344
Epoch35